# 🗓️ QEPC Schedule Viewer

Browse and filter NBA game schedules.

**Features:**
- 📅 View today's games
- 📆 Browse upcoming games
- 🔍 Filter by team, date range
- 📊 View schedule statistics
- 🏀 Quick game lookup

---

## 🧩 Setup & Load Context

In [ ]:
# --- Robust bootstrap to load notebook_header.py no matter where Jupyter started ---

import sys
import importlib.util
from pathlib import Path

# 1) Find the project root: the folder that contains notebook_header.py
cur = Path.cwd()
project_root = None

for _ in range(6):  # walk up a few levels just in case
    if (cur / "notebook_header.py").exists():
        project_root = cur
        break
    cur = cur.parent

if project_root is None:
    raise FileNotFoundError(
        "Could not find notebook_header.py in the current directory or its parents."
    )

# 2) Make sure project root is on sys.path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 3) Load notebook_header.py as a proper module
header_path = project_root / "notebook_header.py"
spec = importlib.util.spec_from_file_location("notebook_header", header_path)
notebook_header = importlib.util.module_from_spec(spec)

# IMPORTANT: register it in sys.modules so @dataclass doesn't break
sys.modules[spec.name] = notebook_header

spec.loader.exec_module(notebook_header)

# 4) Now call qepc_notebook_setup from that module
env = notebook_header.qepc_notebook_setup(run_diagnostics=False)
data_dir = env.data_dir
raw_dir = env.raw_dir

print("✅ QEPC environment initialized")
print("project_root:", project_root)
print("data_dir:", data_dir)
print("raw_dir:", raw_dir)


In [ ]:
import pandas as pd
from datetime import datetime, date, timedelta

# Load schedule
schedule_path = project_root / "data" / "Games.csv"
schedule = pd.read_csv(schedule_path)

# Convert date column to datetime
schedule['gameDate'] = pd.to_datetime(schedule['gameDate'])

print(f"✅ Loaded {len(schedule)} games")
print(f"📅 Date range: {schedule['gameDate'].min().date()} to {schedule['gameDate'].max().date()}")

---

## 📅 Today's Games

In [ ]:
# Get today's date
today = date.today()
print(f"🗓️  Today: {today.strftime('%A, %B %d, %Y')}\n")

# Filter for today
todays_games = schedule[schedule['gameDate'].dt.date == today].copy()

if len(todays_games) == 0:
    print("📭 No games scheduled for today")
else:
    print(f"🏀 {len(todays_games)} game(s) today:\n")
    
    # Display games
    for idx, game in todays_games.iterrows():
        away = game.get('awayTeam', 'TBD')
        home = game.get('homeTeam', 'TBD')
        time = game.get('gameDateTimeEst', 'TBD')
        
        # Format time if it's a timestamp
        if pd.notna(time) and time != 'TBD':
            try:
                time_obj = pd.to_datetime(time)
                time = time_obj.strftime('%I:%M %p ET')
            except:
                pass
        
        print(f"   {away} @ {home}")
        print(f"   🕐 {time}")
        print()

---

## 📆 Upcoming Games (Next 7 Days)

In [ ]:
# Get next 7 days
today = date.today()
next_week = today + timedelta(days=7)

upcoming = schedule[
    (schedule['gameDate'].dt.date > today) & 
    (schedule['gameDate'].dt.date <= next_week)
].copy()

print(f"📅 Upcoming games ({today} to {next_week}): {len(upcoming)} games\n")

if len(upcoming) > 0:
    # Group by date
    upcoming['date_str'] = upcoming['gameDate'].dt.strftime('%A, %B %d')
    
    for date_str, games in upcoming.groupby('date_str', sort=False):
        print(f"\n📅 {date_str} ({len(games)} games)")
        print("─" * 60)
        
        for idx, game in games.iterrows():
            away = game.get('awayTeam', 'TBD')
            home = game.get('homeTeam', 'TBD')
            print(f"   {away} @ {home}")
else:
    print("📭 No games in the next 7 days")

---

## 🔍 Filter by Team

In [ ]:
# Change this to your favorite team!
TEAM_NAME = "Los Angeles Lakers"

print(f"🔍 Searching for {TEAM_NAME} games...\n")

team_games = schedule[
    (schedule['homeTeam'] == TEAM_NAME) | 
    (schedule['awayTeam'] == TEAM_NAME)
].copy()

# Separate into past and future
today = date.today()
past_games = team_games[team_games['gameDate'].dt.date < today]
future_games = team_games[team_games['gameDate'].dt.date >= today]

print(f"📊 {TEAM_NAME} Schedule:")
print(f"   Games played: {len(past_games)}")
print(f"   Upcoming games: {len(future_games)}")
print(f"   Total games: {len(team_games)}\n")

# Show next 10 games
if len(future_games) > 0:
    print(f"🗓️  Next 10 games:\n")
    
    for idx, game in future_games.head(10).iterrows():
        game_date = game['gameDate'].strftime('%b %d')
        away = game['awayTeam']
        home = game['homeTeam']
        
        # Determine if home or away
        if home == TEAM_NAME:
            opponent = away
            location = "vs"
        else:
            opponent = home
            location = "@"
        
        print(f"   {game_date}: {location} {opponent}")
else:
    print("📭 No upcoming games found")

---

## 📊 Schedule Statistics

In [ ]:
print("📊 NBA Schedule Statistics\n")
print("═" * 60)

# Overall stats
print(f"\n📅 Total games in schedule: {len(schedule)}")
print(f"🏀 Teams: {schedule['homeTeam'].nunique()}")
print(f"📆 Date range: {schedule['gameDate'].min().date()} to {schedule['gameDate'].max().date()}")

# Games per team
home_games = schedule['homeTeam'].value_counts()
away_games = schedule['awayTeam'].value_counts()
total_games = home_games.add(away_games, fill_value=0)

print(f"\n📊 Games per team:")
print(f"   Average: {total_games.mean():.0f}")
print(f"   Min: {total_games.min():.0f} ({total_games.idxmin()})")
print(f"   Max: {total_games.max():.0f} ({total_games.idxmax()})")

# Games by month
schedule['month'] = schedule['gameDate'].dt.strftime('%B %Y')
games_by_month = schedule.groupby('month').size().sort_index()

print(f"\n📅 Games by month:")
for month, count in games_by_month.items():
    print(f"   {month}: {count} games")

print("\n" + "═" * 60)

---

## 🎯 Custom Date Range Query

In [ ]:
# Customize these dates!
START_DATE = "2025-12-01"  # YYYY-MM-DD
END_DATE = "2025-12-31"    # YYYY-MM-DD

start = pd.to_datetime(START_DATE)
end = pd.to_datetime(END_DATE)

date_range_games = schedule[
    (schedule['gameDate'] >= start) & 
    (schedule['gameDate'] <= end)
].copy()

print(f"🔍 Games between {START_DATE} and {END_DATE}\n")
print(f"📊 Found {len(date_range_games)} games\n")

if len(date_range_games) > 0:
    # Show summary
    date_range_games['date_only'] = date_range_games['gameDate'].dt.date
    games_per_day = date_range_games.groupby('date_only').size()
    
    print(f"📅 Games per day:")
    print(f"   Average: {games_per_day.mean():.1f}")
    print(f"   Max: {games_per_day.max()} ({games_per_day.idxmax()})")
    print(f"   Min: {games_per_day.min()} ({games_per_day.idxmin()})")
    
    # Show data
    print(f"\n📋 Sample games:")
    display(date_range_games[['gameDate', 'awayTeam', 'homeTeam']].head(20))
else:
    print("📭 No games found in this date range")